Here, you develop the 

In [1]:
import os.path
from scipy import *
import numpy as np
from numpy import *
from numpy import linalg as LA
from scipy import linalg as LA2
import sympy as sympy
import sys as sys
import time
import matplotlib.pyplot as plt
import itertools as it
from IPython.core.display import HTML

sys.path.append('/Users/sashacurcic/SashasDirectory/ANAG/FV_MG/')
from Modules import BasicTools as BT
from Modules import WaveTools as WT
from Modules import PlotTools as PT
from Modules import FFTTools as FFTT
from Modules import OperatorTools as OT
from Modules import GridTransferTools as GTT
from Modules import TestTools as TT
from Modules import SolverTools as ST

display(HTML("<style>pre { white-space: pre !important; }</style>"))
np.set_printoptions( linewidth = 10000, threshold = 100000)

This cell takes all of the input parameters.

In [2]:
nh = 16
refRatio = 2
c = 1.
CFL = 0.5
nt = 32
RK = 4
deriv = 'U'
order = 2

This cell contains the relevant switch information.

In [3]:
if (RK == 1):
    TimeIntegratorFunc = ST.ForwardEuler
else:
    if (RK == 2):
        TimeIntegratorFunc = ST.MidpointMeth
    else:
        TimeIntegratorFunc = ST.RK4

if (deriv == 'U'):
    DiffFunc = ST.Upwind
else:
    DiffFunc = TT.CenterDiff#ST.CenterDiff

This cell instantiates the grid, derivative operators, and Fourier modes.

In [4]:
omega = BT.Grid(nh)
finehalf = list(np.arange(int(nh / refRatio)))# + int(nh / refRatio))
omega.AddPatch(refRatio, finehalf)
degFreed = omega.degFreed
nh_min = omega.nh_min
nh_max = omega.nh_max
t, nt = ST.CalcTime(omega, CFL, c, nt = nt)
derivMat = OT.SpaceDeriv(omega, order, 'deriv')
spatOp = -c * derivMat
strings = omega.strings
timePropOp = LA2.expm(t * spatOp)
omegaF = BT.Grid(nh_max)
wavesF = WT.MakeWaves(omegaF)

YOU'RE USING THE UPDATE.
bounds: [-1.   0.   0.5  1. ]
polyInterp: [ 0.5   0.75 -0.25]

YOU'RE USING THE UPDATE.
bounds: [-1.  -0.5 -0.   1. ]
polyInterp: [-0.25  0.75  0.5 ]



In [5]:
polyInterp = GTT.GhostCellStencil(2, -0.5)
print(polyInterp)
print(type(polyInterp))

YOU'RE USING THE UPDATE.
bounds: [-1.   0.   0.5  1. ]
polyInterp: [ 0.5   0.75 -0.25]

[ 0.5   0.75 -0.25]
<class 'numpy.ndarray'>


This cell runs a test on a fifth-order cell-averaged polynomial interpolation for a ghost cell at the edge of the coarse-fine interface. If it doesn't reach the appropriate order of accuracy, there is an `assert` which force quits.

In [6]:
TT.TestPoly(5, -4.5)

YOU'RE USING THE UPDATE.
bounds: [-7 -6 -5 -4 -3 -2 -1]
YOU'RE USING THE UPDATE.
bounds: [-7 -6 -5 -4 -3 -2 -1]
polyInterp: [-0.22070312  1.31966146 -3.23307292  4.69661458 -3.11914063  1.55664063]

Order 0:
2.0 2.0000000000001315
Error = 6.572520305780927e-12%

Order 1:
-4.5 -4.500000000000384
Error = 8.526512829121202e-12%

Order 2:
13.5 13.500000000001677
Error = 1.242133967699138e-11%

Order 3:
-45.5625 -45.56250000000884
Error = 1.9400058453455465e-11%

Order 4:
164.02499999999998 164.02500000005784
Error = 3.527913452236096e-11%

Order 5:
-615.09375 -615.0937500003911
Error = 6.358099424069244e-11%

Order 6:
2372.5044642857138 2053.2857142883586
Error = -13.45492726368639%



In [7]:
derivMat = OT.SpaceDeriv(omega, order, 'C')
print(derivMat)

YOU'RE USING THE UPDATE.
bounds: [-1.   0.   0.5  1. ]
polyInterp: [ 0.5   0.75 -0.25]

YOU'RE USING THE UPDATE.
bounds: [-1.  -0.5 -0.   1. ]
polyInterp: [-0.25  0.75  0.5 ]

[[-12.  20.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  -8.]
 [-16.   0.  16.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0. -16.   0.  16.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0. -16.   0.  16.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0. -16.   0.  16.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0. -16.   0.  16.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0. -16.   0.  16.   0.   0.   0.   0.   0.   0.   0.   0.   0.

In [8]:
omega2 = BT.Grid(nh)
finehalf2 = list(np.arange(int(nh / refRatio)) + int(nh / refRatio))
omega2.AddPatch(refRatio, finehalf2)

In [9]:
derivMat = OT.SpaceDeriv(omega2, order, 'C')
print(derivMat)

YOU'RE USING THE UPDATE.
bounds: [-1.   0.   0.5  1. ]
polyInterp: [ 0.5   0.75 -0.25]

YOU'RE USING THE UPDATE.
bounds: [-1.  -0.5 -0.   1. ]
polyInterp: [-0.25  0.75  0.5 ]

[[  0.   8.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  -4.  -4.]
 [ -8.   0.   8.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.  -8.   0.   8.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.  -8.   0.   8.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.  -8.   0.   8.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.  -8.   0.   8.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.  -8.   0.   8.   0.   0.   0.   0.   0.   0.   0.   0.   0.